In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from scipy.stats import mode

from sklearn.datasets import load_digits
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.cluster import DBSCAN, MiniBatchKMeans, KMeans, AgglomerativeClustering

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd

from time import time
import os
import pathlib
import shutil
from skimage import io

In [ ]:
# Loading the dataset
df = pd.read_csv('data/train_complete.csv')
df.head()

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util

In [ ]:
# isolate only images that have 0 or 1 defect
util.isolate_single_defects(df)

In [ ]:
path = pathlib.Path.cwd()
path

In [ ]:
#
os.mkdir(path.joinpath('data','single_defect_train_images'))

In [ ]:
# von Michael kopiert für Ordnererstellung basierend auf x_train
for i in range(len(df)):
    origin_train_path = path.joinpath('data', 'train_images')
    source_file = df.iloc[i,1]
    target_directory = path.joinpath('data', 'single_defect_train_images')
    shutil.copy2(origin_train_path.joinpath(source_file) , target_directory.joinpath(source_file))

In [ ]:
image = io.imread('data/single_defect_train_images/0002cc93b.jpg')
image.shape

In [ ]:
# Creating the KMeans model and predict classes (n_clusters =5 ; due to 5 classes)
kmeans = KMeans(n_clusters=5, random_state=0)
clusters = kmeans.fit_predict(digits.data)
kmeans.cluster_centers_.shape

In [ ]:
# Create a labels array to match the learned cluster lables with the true labels
labels = np.zeros_like(clusters)
for i in range(5):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

In [ ]:
# Plotting the confusing matrix
mat = confusion_matrix(digits.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

print('Accuracy: ', accuracy_score(digits.target, labels))
print('==============================================================')
print(confusion_matrix(digits.target,labels))
print('==============================================================')
print(classification_report(digits.target,labels))